# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
os.chdir('..')

from nn.models_tf2 import Models
from botbidder import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'), False)  # Load sampling strategies



Loading config file d:\GitHub\ben\src\./config/default.conf
Loading config file d:\GitHub\ben\src\./config/default.conf


### Running through an example

In [2]:

vuln_ns, vuln_ew = False, False

from ddsolver import ddsolver
dds = ddsolver.DDSolver()

# you sit West and hold:
hand = '6.AKJT82.762.K63'

# the auction goes:
auction = ['1D', '3S']
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, seat=2, dealer=0, ddsolver=dds, bba_is_controlling=False, verbose=False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '4H',
  'insta_score': 1.003,
  'alert': 'False',
  'explanation': 'Bidable suit -- 6+!H; 6-!S; 11-24 HCP'},
 {'call': 'PASS', 'insta_score': 0.179}]

In [3]:
auction = ['1D', '3S', '4H', '4S', '5H', 'PASS']
bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

[{'call': 'PASS',
  'insta_score': 1.178,
  'adjustment': 0,
  'expected_score': 265,
  'expected_tricks': 10.8,
  'alert': 'False',
  'explanation': ' -- 6+!H; 6-!S; 11-21 HCP'},
 {'call': '6H', 'insta_score': 0.016}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [4]:
bid.samples

['x.Qxxx.AKQTxx.xx AQJTxxx.xx.xx.xx x.AKJTxx.xxx.Kxx Kxxx.x.Jx.AQJTxx - 0.72722',
 'x.Qxxx.AKJxxx.Jx AKJTxxx.xx.xx.Tx x.AKJTxx.xxx.Kxx Qxxx.x.QT.AQxxxx - 0.72617',
 '.Qxxx.AKJTxx.Jxx QJxxxxx.xxx.Q.Qx x.AKJTxx.xxx.Kxx AKTxx..xxx.ATxxx - 0.72337',
 'x.Qxxx.AKQxxx.Tx KJTxxxx.xx.Jx.Ax x.AKJTxx.xxx.Kxx AQxx.x.Tx.QJxxxx - 0.71872',
 'x.Qxxx.AKQxxx.Tx QTxxxxx.xx.J.Axx x.AKJTxx.xxx.Kxx AKJx.x.Txx.QJxxx - 0.71708',
 'x.Qxxx.KQTxxx.Ax AQTxxxx..xx.Jxxx x.AKJTxx.xxx.Kxx KJxx.xxx.AJ.QTxx - 0.71445',
 'K.Qxxx.KQJxx.QJx QJTxxxx.x.Axx.xx x.AKJTxx.xxx.Kxx Axxx.xx.Tx.ATxxx - 0.71434',
 '.Qxxx.AKJTxx.xxx KQTxxxx.xx.Qxx.x x.AKJTxx.xxx.Kxx AJxxx.x.x.AQJTxx - 0.71278',
 '.Qxx.AKJxxx.Jxxx KQJTxxx.x.Txx.xx x.AKJTxx.xxx.Kxx Axxxx.xxx.Q.AQTx - 0.71135',
 '.Qxxx.AKQTx.xxxx AKJTxxx.x.Jxx.xx x.AKJTxx.xxx.Kxx Qxxxx.xx.xx.AQJT - 0.71079',
 '.Qxx.AKJxxx.Jxxx AKJxxxx.xxx.Tx.T x.AKJTxx.xxx.Kxx QTxxx.x.Qx.AQxxx - 0.71004',
 '.xxx.AKQxxx.QTxx AQxxxxx.xx.J.Jxx x.AKJTxx.xxx.Kxx KJTxx.Qx.Txx.Axx - 0.70935',
 'J.Qxxx.AKJTxx.

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [5]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(['PAD_START','2N','PASS','3C','PASS','3D'], 0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :], models.n_cards_bidding)} {hand_to_str(sample_hands[i, 1, :], models.n_cards_bidding)} {hand_to_str(sample_hands[i, 2, :], models.n_cards_bidding)} {hand_to_str(sample_hands[i, 3, :], models.n_cards_bidding)}')

Good quality: 0.8874763110104729
x.AKJTxx.xxx.Kxx KTx.Qxx.AKQx.AQx Jxxxx..JTxx.Txxx AQxx.xxxx.xx.Jxx
x.AKJTxx.xxx.Kxx AKx.Qxx.AKxx.ATx xxxxx.x.Jxx.Jxxx QJTx.xxx.QTx.Qxx
x.AKJTxx.xxx.Kxx Kxx.Qxx.AKQx.AQx Txxxx.x.JTx.Txxx AQJx.xxx.xxx.Jxx
x.AKJTxx.xxx.Kxx AKx.xxx.AKJx.AJx JTxxx.Qx.xxx.xxx Qxxx.xx.QTx.QTxx
x.AKJTxx.xxx.Kxx AKx.xxx.AKx.AQxx Txxxx.x.JTxx.JTx QJxx.Qxx.Qxx.xxx
x.AKJTxx.xxx.Kxx AKT.Qxx.AQT.AJxx Jxxxx.x.Jxxx.Txx Qxxx.xxx.Kxx.Qxx
x.AKJTxx.xxx.Kxx AQT.xxx.AKxx.AQJ KJxxx.x.xx.Txxxx xxxx.Qxx.QJTx.xx
x.AKJTxx.xxx.Kxx KQx.Qxx.AKx.AQTx xxxxx.x.Qxx.Jxxx AJTx.xxx.JTxx.xx
x.AKJTxx.xxx.Kxx AKx.xxx.AKJx.AJx QTxxx.x.Txxx.Qxx Jxxx.Qxx.Qx.Txxx
x.AKJTxx.xxx.Kxx AKx.Qx.KQxx.AQxx Txxxx.x.Txxx.xxx QJxx.xxxx.AJ.JTx
x.AKJTxx.xxx.Kxx AKx.Qxx.AJx.AQxx xxxxx.xx.QTx.Txx QJTx.xx.Kxxx.Jxx
x.AKJTxx.xxx.Kxx AKx.Qx.AKxx.ATxx Jxxxx.x.Txx.Jxxx QTxx.xxxx.QJx.Qx
x.AKJTxx.xxx.Kxx AKx.xx.AKxx.AQTx Txxxx.x.JTx.Jxxx QJxx.Qxxx.Qxx.xx
x.AKJTxx.xxx.Kxx AKx.xx.AKxx.AQxx QTxxx.x.JTxx.Txx Jxxx.Qxxx.Qx.Jxx
x.AKJTxx.xxx.Kx

How do we know on which sample 4H is making and on which sample it is not? It's the topic for another tutorial, but basically we could do either (1) solve the sample double dummpy or (2) estimate the outcome using a neural network

### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

In [6]:
hands = random_deal().split()
hands # N, E, S, W

['84.832.KQ73.J872',
 'K9.AQJ9754.82.Q9',
 'AQJT72.KT.AJ6.64',
 '653.6.T954.AKT53']

In [7]:
bidder_bots = [BotBid([False, False], hand, models, sampler,i, 0, dds, False, False) for i, hand in enumerate(hands)]

In [8]:
auction = []  # since North deals, we don't need any 'PAD_START'

turn_i = 0  # whose turn is it to bid

while not bidding.auction_over(auction):
    auction.append(bidder_bots[turn_i].bid(auction).bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    
auction

['PASS',
 '1H',
 '1S',
 'X',
 'PASS',
 '2H',
 '2S',
 'PASS',
 'PASS',
 '3H',
 'PASS',
 'PASS',
 'PASS']